<a href="https://colab.research.google.com/github/vinothalagupandi/keras-tf/blob/master/classroom-pracrtice/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget --header 'Host: storage.googleapis.com' --user-agent 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://www.kaggle.com/' --header 'Upgrade-Insecure-Requests: 1' --header 'Sec-Fetch-Dest: document' --header 'Sec-Fetch-Mode: navigate' --header 'Sec-Fetch-Site: cross-site' --header 'Sec-Fetch-User: ?1' 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10338/862042/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1707672971&Signature=Y9M82nFK7KOEYvwG3M229IiD7K07YqETpjqOGrDh2MnJcJFCrGiYcDjOx9flDj8Wj72GBI4OqYMyadyn0FvR5kr5YVxe2k%2BONJyE5PkF%2B%2FtuttqaxPD4UxnomYtZL8xuyO6ve%2BiSVO5%2FB4l6HtoUz%2FKXSNbZ%2BRRJFgBHA4jfZe9oxf0VxvR6YtOdGLe3iHv9J%2FlsymZxeS4yGVLSajupcAHtHgo%2F%2B%2FnuPp1OSWMZhym%2BnqVUeJ1ait9DAwv4Fg4cnqUdrpsUuNmCitwaZe97wefu0KLiEoqyGFx3FcWPzyhsu4pl%2F55hZFD2GUMNXMfG%2BD6VOE3F4HvIccgUf1EkxA%3D%3D&response-content-disposition=attachment%3B+filename%3Drsna-pneumonia-detection-challenge.zip' --output-document 'rsna-pneumonia-detection-challenge.zip'

--2024-02-08 18:47:44--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10338/862042/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1707672971&Signature=Y9M82nFK7KOEYvwG3M229IiD7K07YqETpjqOGrDh2MnJcJFCrGiYcDjOx9flDj8Wj72GBI4OqYMyadyn0FvR5kr5YVxe2k%2BONJyE5PkF%2B%2FtuttqaxPD4UxnomYtZL8xuyO6ve%2BiSVO5%2FB4l6HtoUz%2FKXSNbZ%2BRRJFgBHA4jfZe9oxf0VxvR6YtOdGLe3iHv9J%2FlsymZxeS4yGVLSajupcAHtHgo%2F%2B%2FnuPp1OSWMZhym%2BnqVUeJ1ait9DAwv4Fg4cnqUdrpsUuNmCitwaZe97wefu0KLiEoqyGFx3FcWPzyhsu4pl%2F55hZFD2GUMNXMfG%2BD6VOE3F4HvIccgUf1EkxA%3D%3D&response-content-disposition=attachment%3B+filename%3Drsna-pneumonia-detection-challenge.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.207, 64.233.170.207, 142.251.175.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3932287530 (3.7G) [application/zip]
Saving to: ‘rsna-

In [ ]:
!unzip /content/rsna-pneumonia-detection-challenge.zip

# Chest X ray train test val data exploration

In [ ]:

import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("/content/stage_2_train_labels.csv")

# Keep only the "patientId" and "Target" columns
df = df[["patientId", "Target"]]

# Print the DataFrame
print(df.head())


In [ ]:

df = df.drop_duplicates()
df = df[df.notnull().all(axis=1)]
print(df.head())


In [ ]:

import os

# Get the path to the train images directory
train_images_dir = "/content/stage_2_train_images"

# List the files in the directory
files = os.listdir(train_images_dir)

# Print the files
print(files[:10])


In [ ]:
len(df),len(files)

In [ ]:
"0004cfab-14fd-4e49-80ba-63a80b6bddd6.dcm" in files

In [ ]:
for ids in list(df["patientId"]):
  if f"{ids}.dcm" not in files:
    print(f"{id} is not heree")

In [ ]:

from sklearn.model_selection import train_test_split

# Split the DataFrame into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)
# Print the shapes of the train and validation DataFrames
print(f"Train DataFrame shape: {train_df.shape}")
print(f"Validation DataFrame shape: {val_df.shape}")


In [ ]:
!pip install pydicom

In [ ]:

import tensorflow as tf
import pydicom
def read_dicom_image(path,label):
    img_raw = tf.io.read_file(path)
    image = tfio.image.decode_dicom_image(img_raw, dtype=tf.uint16)
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.grayscale_to_rgb(image,name=None)
    label = tf.reshape(label,[1])
    return image,label

def create_dataset(df, image_dir):
    paths = [os.path.join(image_dir, f"{id}.dcm") for id in df["patientId"]]
    labels = list(df["Target"].values)
    paths=tf.constant(paths)
    labels = tf.constant(labels)
    dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
    dataset = dataset.map(read_dicom_image)
    return dataset
train_images_dir = "/content/stage_2_train_images"

train_dataset = create_dataset(train_df, train_images_dir,batch_size=64)
val_dataset = create_dataset(val_df, train_images_dir,64)

In [ ]:
for x,y in train_dataset:
  print(y)
  # print(x.shape,y.shape)
  # print(x.numpy(),y.numpy())
  break

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

import matplotlib.pyplot as plt

# Get the first image and label from the train dataset
image, label = next(iter(train_dataset))

# Print the image shape and label
print(f"Image shape: {image.shape}")
print(f"Label: {label}")

# Plot the image
plt.imshow(image[0])
plt.show()


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers


In [ ]:

from tensorflow.keras.applications import ResNet50
# Load the ResNet50 model with pre-trained weights
conv_base = ResNet50(weights="imagenet",include_top=False,input_shape=(224, 224, 3))
conv_base.trainable = False
# Print the model summary
# conv_base.summary()


In [ ]:

x = keras.applications.resnet50.preprocess_input(image)
x.shape


In [ ]:
inputs= keras.Input(shape=(224,224, 3))
x = keras.applications.resnet50.preprocess_input(inputs)
x = conv_base(inputs)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)


In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="rsna_resnet50.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    callbacks=callbacks)

In [ ]:
# prompt: Tensor("args_0:0", shape=(), dtype=string)  in dataset

paths = tf.constant(["/content/stage_2_train_images/0004cfab-14fd-4e49-80ba-63a80b6bddd6.dcm", "/content/stage_2_train_images/0031985d-9d1a-4b83-b833-f818b32975b8.dcm"])
labels = tf.constant([1, 0])
dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
dataset = dataset.map(lambda x, y: (read_dicom_image(x), y))

for x,y in dataset:
  print(x.shape,y)
